In [ ]:
import json
import trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

In [ ]:
import trt_pose.models

num_parts = len(human_pose['keypoints'])
num_links = len(human_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()

In [ ]:
import torch

WIDTH = 224
HEIGHT = 224

data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

In [ ]:
import torch2trt
OPTIMIZED_MODEL = 'resnet18_baseline_att_224x224_A_epoch_249_trt.pth'

from torch2trt import TRTModule
model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

In [ ]:
import time

#t0 = time.time()
#torch.cuda.current_stream().synchronize()
#for i in range(50):
#    y = model_trt(data)
#torch.cuda.current_stream().synchronize()
#t1 = time.time()

#print(50.0 / (t1 - t0))

In [ ]:
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

In [ ]:
#from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
#from jetcam.utils import bgr8_to_jpeg

#camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=15)

camera.running = True

In [ ]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_w = ipywidgets.Image(format='jpeg', width=WIDTH, height=HEIGHT)

display(image_w)

In [ ]:
import numpy as np
import shutil
global folder
camera.running = True
folder = "/home/jetbot/Desktop/"
file_path = folder + "temp.csv"
f = open(file_path, "a")

In [ ]:
def get_keypoint(humans, hnum, peaks):
    #check invalid human index
    kpoint = []
    human = humans[0][hnum]
    C = human.shape[0]
    for j in range(C):
        k = int(human[j])
        if k >= 0:
            peak = peaks[0][j][k]   # peak[1]:width, peak[0]:height
            peak = [j, float(peak[0]), float(peak[1])]
            kpoint.append(peak)
            #print('index:%d : success [%5.3f, %5.3f]'%(j, peak[1], peak[2]) )
        else:    
            peak = [j, -1, -1] # -1 = None
            kpoint.append(peak)
            #print('index:%d : None %d'%(j, k) )
    return kpoint

In [ ]:
from threading import Thread
import os
def upload():
    global upload_running
    upload_running = 1
    # for this to work we need to exchange credentials manually first and install sshpass
    # change password for the real password, user for the real username and ip for the real ip or domain
    # change the path after ip_of_server: to match the real path where you want to save the files
    cmd = "sshpass \"password\"scp /home/jetbot/Desktop/data/*1*.csv username@ip_of_server:/home/jetbot/data/"
    ret = os.system(cmd)
    if ret != 0:
        print(str(int(time.time()))+ " upload returned error code: " + str(ret))
    upload_running = 0
    files_copied = 0
    

In [ ]:
def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    keypoints = get_keypoint(objects, 0, peaks) # 0 for just the first human recognized in the frame
    # print(keypoints) # print keypoints for first human recognized in the frame
    draw_objects(image, counts, objects, peaks)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    np.savetxt(f, keypoints, delimiter=',', fmt='%.4f', newline='\n')
    global kp_sets_saved
    global files_copied
    kp_sets_saved = kp_sets_saved + 1
    print("kp sets saved: " + str(kp_sets_saved))
    if kp_sets_saved == 100: # save every 100 sets of keypoints
        current_epoch = int(time.time())
        new_file = folder + str(current_epoch) + ".csv"
        print(str(int(time.time())) + " temp file copied to: " + new_file)
        shutil.copy(file_path, new_file)
        files_copied = files_copied + 1
        kp_sets_saved = 0
        print(str(int(time.time())) + " temp file truncated")
        f.truncate()
    #if files_copied == 20 and upload_running == 0:
        # disabled until tested
        # thread = Thread(target = threaded_function, args = (10, )) 
        # thread.start()

In [ ]:
kp_sets_saved = 0
files_copied = 0
upload_running = 0
execute({'new': camera.value})

In [ ]:
camera.observe(execute, names='value')

In [ ]:
camera.unobserve_all()

In [ ]:
f.close()